In [2]:
"""
The code for testing the classifiers. Uses binary.csv and multiclass.csv

Loads feature lists and tuning ranges from pickles in CWD.

Change model types in models to tune on specific ones.

Writes output files.

Author: Wesley
"""

# Accelerates tuning of some classifiers
# from sklearnex import patch_sklearn
# patch_sklearn()

import pandas as pd
import numpy as np

from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real

from sklearn.feature_selection import RFECV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import pickle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score
)

from time import time

from sklearn.metrics import classification_report

In [3]:
binary = pd.read_csv("binary.csv")
multiclass = pd.read_csv("multiclass.csv")

Preprocessing (make labels numeric)

In [4]:
# Binarize binary dataset.
binary[" Label"] = binary[" Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

print("The text labels from the encoding will be passed to classification report so we can interpret our results more easily.\n")
binary_labels = ["BENIGN", "ATTACK"]

print("Binary Label Encodings (in order of digits 0 -> 1): ")
print(binary_labels)

# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()
multiclass[" Label"] = le.fit_transform(multiclass[" Label"].values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multiclass[' Label']))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)

The text labels from the encoding will be passed to classification report so we can interpret our results more easily.

Binary Label Encodings (in order of digits 0 -> 1): 
['BENIGN', 'ATTACK']

Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load feature sets and search spaces and enumerate their contents.

In [5]:
feature_sets = pickle.load(open("feature_sets.pickle", 'rb'))
search_spaces = pickle.load(open("hyperparameter_search_spaces.pickle", 'rb'))

print(f"Available Tuning Ranges: {search_spaces.keys()}")

print("Feature Sets for Binary Dataset:")
for key, value in feature_sets["Binary"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")
        
    else:
        print(key)

print("Feature Sets for Multiclass Dataset:")
for key, value in feature_sets["Multiclass"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")

    else:
        print(key)

Available Tuning Ranges: dict_keys(['XGBoost', 'Bagging SVM', 'SVC (RBF)', 'SVC (Poly)', 'Logistic Regression', 'Random Forest', 'KNN', 'Linear SVC', 'Naive Bayes', 'Decision Tree'])
Feature Sets for Binary Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Linear SVC', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95
Feature Sets for Multiclass Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95


In [6]:
"""
This is a helper method to place our performance results in a DataFrame for future analysis.
"""
def format_results(y_test, predicted_values, fold_index, fitTime):
    # get scores
    accuracy = accuracy_score(y_test,predicted_values)
    recall_pos = recall_score(y_test, predicted_values)
    precision_pos = precision_score(y_test,predicted_values)
    f1 = f1_score(y_test,predicted_values)
    tn, fp, fn, tp = confusion_matrix(y_test, predicted_values).ravel()

    # This avoids divide by zero errors in some cases of no predicted samples.
    if (tn + fp) > 0:
        recall_neg = tn / (tn + fp)
    else:
        recall_neg = 0

    if (tn + fn) > 0:
        precision_neg = tn / (tn + fn)
    else:
        precision_neg = 0

    cols = ["Fitting Time", "accuracy", "TP", "TN", "FP", "FN", "Precision: 0", "Precision: 1", "Recall: 0", "Recall: 1", "F1 Score"]
    results = [fitTime, accuracy, tp, tn, fp, fn, precision_neg, precision_pos, recall_neg, recall_pos, f1]

    outFrame = pd.DataFrame([results], columns=cols, index=[fold_index])

    return outFrame

In [7]:
"""
This is a helper method to place our performance results in a DataFrame for future analysis.
"""
def format_results_multiclass(y_test, predicted_values, fold_index, fitTime):
    # get scores
    accuracy = accuracy_score(y_test,predicted_values)
    recall_pos = recall_score(y_test, predicted_values, average='macro')
    precision_pos = precision_score(y_test,predicted_values, average='macro')
    f1 = f1_score(y_test,predicted_values, average='macro')

    cols = ["Fitting Time", "accuracy", "Precision", "Recall", "F1 Score"]
    results = [fitTime, accuracy, precision_pos, recall_pos, f1]

    outFrame = pd.DataFrame([results], columns=cols, index=[fold_index])

    return outFrame

This is for tuning on the Binary set.

In [8]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42),
    "Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
}

score_methods = ['accuracy']

feature_set = feature_sets["Binary"]

y = binary[" Label"].copy()
X = binary.drop([" Label"], axis=1)

In [8]:
params = []
estimators = []

# This will hold all of our results.
runFrame = None

for name, model in models.items():
        for feature_key, feature_val in feature_set.items():

            # If we're on the RFE sets, check if we have one for this classifier. If not, skip it.
            if feature_key == "RFE Sets":
                if name in feature_val.keys():
                    feature_val = feature_val[name]
                else:
                    continue
            
            for score_method in score_methods:
                opt = BayesSearchCV(estimator=model,search_spaces=search_spaces[name],n_iter=30,scoring=score_method,cv=5,n_jobs=5)
                kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

                counter = 0

                # Used to hold data for a single run (performance metric)
                perfFrame = None

                for train_index, test_index in kf.split(X, y):

                    counter += 1

                    # PCA requires different logic to create X.
                    if feature_key != "PCA":
                        current_X = X.loc[:, feature_val]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
                        
                    else:
                        pca_trans = PCA(n_components=feature_val, random_state=42)
                        current_X = X.loc[:, X.columns]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        # Apply PCA to training set and use it to transform test set.
                        X_train = pca_trans.fit_transform(X_train)
                        X_test = pca_trans.transform(X_test)

                        # Convert back to DataFrames
                        pca_cols = ["PC"+str(i) for i in list(range(1, len(X_train[0])+1))]
                        X_train = pd.DataFrame(data=X_train, columns=pca_cols)
                        X_test = pd.DataFrame(data=X_test, columns=pca_cols)

                    startTime = time()

                    opt.fit(X_train,Y_train)

                    endTime = time()
                    fitTime = endTime - startTime

                    predicted_values = opt.predict(X_test)

                    # get metrics for this fold.
                    foldFrame = format_results(Y_test, predicted_values, counter, fitTime)

                    # Add them to our lists of metric.
                    if perfFrame is None:
                        perfFrame = foldFrame
                    else:
                        perfFrame = pd.concat([perfFrame, foldFrame])

                    # Print a classification report as well.
                    print(classification_report(Y_test, predicted_values, target_names=binary_labels, digits=6))

                    # Add tuple with the best params as well as the related model/config
                    params.append((f"Binary {name} {feature_key} {score_method} Fold {counter}", opt.best_params_))
                    estimators.append((f"Binary {name} {feature_key} {score_method} Fold {counter}", opt.best_estimator_))

                # Create a new line in the results table that averages all the folds
                perfFrame.loc["fold average"] = perfFrame.mean()

                # Mark the results table with the chosen classifier and the current performance metric.
                perfFrame['metric'] = [score_method for j in range(0,6)]
                perfFrame['Classifier'] = [name for j in range(0,6)]
                perfFrame['Feature Set'] = [feature_key for j in range(0,6)]
                perfFrame['Dataset'] = ["binary" for j in range(0,6)]
                print(f"{name} with {feature_key} and {score_method} completed.")

                # Add this run to the table with all runs.
                if runFrame is None:
                    runFrame = perfFrame
                else:
                    runFrame = pd.concat([runFrame, perfFrame])

# Write output file and list of best estimators and parameters to be used later.
runFrame.to_csv(f"binary_results{time()}.csv")
pickle.dump(params, open(f"binary_params{time()}.pickle", "wb"))
pickle.dump(estimators, open(f"binary_estimators{time()}.pickle", "wb"))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

              precision    recall  f1-score   support

      BENIGN   0.997149  0.993220  0.995181     10915
      ATTACK   0.993246  0.997160  0.995199     10914

    accuracy                       0.995190     21829
   macro avg   0.995197  0.995190  0.995190     21829
weighted avg   0.995198  0.995190  0.995190     21829



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

              precision    recall  f1-score   support

      BENIGN   0.996690  0.993311  0.994998     10914
      ATTACK   0.993335  0.996702  0.995015     10915

    accuracy                       0.995007     21829
   macro avg   0.995012  0.995007  0.995007     21829
weighted avg   0.995012  0.995007  0.995007     21829



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

...

              precision    recall  f1-score   support

      BENIGN   0.686268  0.995510  0.812458     10914
      ATTACK   0.991828  0.544896  0.703371     10914

    accuracy                       0.770203     21828
   macro avg   0.839048  0.770203  0.757914     21828
weighted avg   0.839048  0.770203  0.757914     21828



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

              precision    recall  f1-score   support

      BENIGN   0.686191  0.993953  0.811885     10914
      ATTACK   0.989035  0.545446  0.703124     10914

    accuracy                       0.769699     21828
   macro avg   0.837613  0.769699  0.757504     21828
weighted avg   0.837613  0.769699  0.757504     21828



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

              precision    recall  f1-score   support

      BENIGN   0.687278  0.994961  0.812982     10914
      ATTACK   0.990876  0.547279  0.705112     10914

    accuracy                       0.771120     21828
   macro avg   0.839077  0.771120  0.759047     21828
weighted avg   0.839077  0.771120  0.759047     21828

Naive Bayes with PCA and accuracy completed.


This is for tuning on the multiclass set

In [9]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42, num_class=11, objective='multi:softmax'),
    "Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
}

score_methods = ['accuracy']

feature_set = feature_sets["Multiclass"]

y = multiclass[" Label"].copy()
X = multiclass.drop([" Label"], axis=1)

In [12]:
params = []
estimators = []

# This will hold all of our results.
runFrame = None

for name, model in models.items():
        for feature_key, feature_val in feature_set.items():

            # If we're on the RFE sets, check if we have one for this classifier. If not, skip it.
            if feature_key == "RFE Sets":
                if name in feature_val.keys():
                    feature_val = feature_val[name]
                else:
                    continue
            
            for score_method in score_methods:
                opt = BayesSearchCV(estimator=model,search_spaces=search_spaces[name],n_iter=50,scoring=score_method,cv=5,n_jobs=5)
                kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

                counter = 0

                # Used to hold data for a single run (performance metric)
                perfFrame = None

                for train_index, test_index in kf.split(X, y):

                    counter += 1

                    # PCA requires different logic to create X.
                    if feature_key != "PCA":
                        current_X = X.loc[:, feature_val]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
                        
                    else:
                        pca_trans = PCA(n_components=feature_val, random_state=42)
                        current_X = X.loc[:, X.columns]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        # Apply PCA to training set and use it to transform test set.
                        X_train = pca_trans.fit_transform(X_train)
                        X_test = pca_trans.transform(X_test)

                        # Convert back to DataFrames
                        pca_cols = ["PC"+str(i) for i in list(range(1, len(X_train[0])+1))]
                        X_train = pd.DataFrame(data=X_train, columns=pca_cols)
                        X_test = pd.DataFrame(data=X_test, columns=pca_cols)

                    startTime = time()

                    opt.fit(X_train,Y_train)

                    endTime = time()
                    fitTime = endTime - startTime

                    predicted_values = opt.predict(X_test)

                    # get metrics for this fold.
                    foldFrame = format_results_multiclass(Y_test, predicted_values, counter, fitTime)

                    # Add them to our lists of metric.
                    if perfFrame is None:
                        perfFrame = foldFrame
                    else:
                        perfFrame = pd.concat([perfFrame, foldFrame])

                    # Print a classification report as well.
                    print(classification_report(Y_test, predicted_values, target_names=multiclass_labels, digits=6))

                    # Add tuple with the best params as well as the related model/config
                    params.append((f"Multiclass {name} {feature_key} {score_method} Fold {counter}", opt.best_params_))
                    estimators.append((f"Multiclass {name} {feature_key} {score_method} Fold {counter}", opt.best_estimator_))

                # Create a new line in the results table that averages all the folds
                perfFrame.loc["fold average"] = perfFrame.mean()

                # Mark the results table with the chosen classifier and the current performance metric.
                perfFrame['metric'] = [score_method for j in range(0,6)]
                perfFrame['Classifier'] = [name for j in range(0,6)]
                perfFrame['Feature Set'] = [feature_key for j in range(0,6)]
                perfFrame['Dataset'] = ["multiclass" for j in range(0,6)]
                print(f"{name} with {feature_key} and {score_method} completed.")

                # Add this run to the table with all runs.
                if runFrame is None:
                    runFrame = perfFrame
                else:
                    runFrame = pd.concat([runFrame, perfFrame])

# Write output file, best parameters, and best models to be used later.
runFrame.to_csv(f"multiclass_results{time()}.csv")
pickle.dump(params, open(f"multiclass_params{time()}.pickle", "wb"))
pickle.dump(estimators, open(f"multiclass_estimators{time()}.pickle", "wb"))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


              precision    recall  f1-score   support

         DNS   0.743430  0.502702  0.599814     10917
        LDAP   0.579027  0.629260  0.603099     10916
       MSSQL   0.924846  0.950348  0.937424     10916
         NTP   0.993335  0.996702  0.995016     10916
     NetBIOS   0.953845  0.986350  0.969825     10916
        SNMP   0.684661  0.827150  0.749191     10917
        SSDP   0.498920  0.444485  0.470132     10916
         Syn   0.599956  0.988366  0.746669     10916
        TFTP   0.998158  0.992671  0.995407     10916
         UDP   0.509311  0.573745  0.539611     10916
     UDP-lag   0.790740  0.240909  0.369304     10917

    accuracy                       0.739330    120079
   macro avg   0.752384  0.739335  0.725045    120079
weighted avg   0.752384  0.739330  0.725041    120079



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


              precision    recall  f1-score   support

         DNS   0.739440  0.503573  0.599128     10916
        LDAP   0.577339  0.626363  0.600852     10917
       MSSQL   0.933194  0.943111  0.938126     10916
         NTP   0.993341  0.997527  0.995429     10916
     NetBIOS   0.956046  0.984335  0.969984     10916
        SNMP   0.678746  0.827043  0.745592     10916
        SSDP   0.499488  0.446734  0.471641     10917
         Syn   0.597303  0.982136  0.742837     10916
        TFTP   0.996962  0.992213  0.994582     10916
         UDP   0.510824  0.568523  0.538131     10916
     UDP-lag   0.768079  0.249084  0.376176     10916

    accuracy                       0.738237    120078
   macro avg   0.750069  0.738240  0.724771    120078
weighted avg   0.750066  0.738237  0.724768    120078

              precision    recall  f1-score   support

         DNS   0.750000  0.513100  0.609334     10916
        LDAP   0.583838  0.618175  0.600516     10916
       MSSQL   0.922562 

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


              precision    recall  f1-score   support

         DNS   0.738619  0.500916  0.596976     10916
        LDAP   0.576979  0.623065  0.599137     10917
       MSSQL   0.931112  0.947233  0.939104     10916
         NTP   0.993251  0.997710  0.995476     10916
     NetBIOS   0.956808  0.984243  0.970332     10916
        SNMP   0.679367  0.829791  0.747082     10916
        SSDP   0.500923  0.397912  0.443514     10917
         Syn   0.596271  0.987266  0.743498     10916
        TFTP   0.997329  0.992122  0.994719     10916
         UDP   0.507873  0.617534  0.557361     10916
     UDP-lag   0.782583  0.242030  0.369717     10916

    accuracy                       0.738162    120078
   macro avg   0.751010  0.738166  0.723356    120078
weighted avg   0.751007  0.738162  0.723353    120078

              precision    recall  f1-score   support

         DNS   0.748005  0.515299  0.610219     10916
        LDAP   0.583118  0.620191  0.601083     10916
       MSSQL   0.924298 

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


              precision    recall  f1-score   support

         DNS   0.692218  0.490564  0.574201     10916
        LDAP   0.568118  0.595181  0.581335     10916
       MSSQL   0.898887  0.917827  0.908259     10916
         NTP   0.900460  0.932301  0.916104     10916
     NetBIOS   0.951274  0.978199  0.964549     10917
        SNMP   0.667316  0.815317  0.733930     10916
        SSDP   0.495329  0.500275  0.497790     10916
         Syn   0.595354  0.993130  0.744438     10917
        TFTP   0.993640  0.987541  0.990581     10916
         UDP   0.513654  0.532429  0.522873     10916
     UDP-lag   0.892585  0.204012  0.332115     10916

    accuracy                       0.722439    120078
   macro avg   0.742622  0.722434  0.706016    120078
weighted avg   0.742622  0.722439  0.706018    120078

              precision    recall  f1-score   support

         DNS   0.597521  0.631458  0.614021     10916
        LDAP   0.592507  0.423049  0.493640     10916
       MSSQL   0.903013 